In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
import pickle

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pylab as pylab

In [5]:
t0 = time.time()

c = cdsapi.Client()
r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'source': 'IGRA_H',
        'area': [
            90, -180, -90,
            180,
        ],
        'format': 'csv-lev.zip',
        'variable': 'air_temperature',
        'pressure_level':500,
        'day': [ '01', ],
        'month': '01',
        'year': '2001',
    },
    'download.csv-lev.zip')
if True:
    # Start Download
    r.download(target='download.zip')
    # Check file size
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    
print("Time elapsed: ", time.time()-t0, "s")

/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2020-11-27 12:08:09,529 INFO Welcome to the CDS
2020-11-27 12:08:09,530 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usag

Time elapsed:  2.944047212600708 s


In [6]:
t0 = time.time()

c = cdsapi.Client()
r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'source': 'IGRA_H',
        'area': [
            90, -180, -90,
            180,
        ],
        'format': 'csv-lev.zip',
        'variable': 'air_temperature',
        'pressure_level':500,
        'day': [ '02', ],
        'month': '01',
        'year': '2001',
    },
    'download.csv-lev.zip')
if True:
    # Start Download
    r.download(target='download.zip')
    # Check file size
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    
print("Time elapsed: ", time.time()-t0, "s")

/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2020-11-27 12:08:57,164 INFO Welcome to the CDS
2020-11-27 12:08:57,165 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usag

Time elapsed:  2.925114393234253 s


In [10]:
t0 = time.time()

c = cdsapi.Client()
r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'source': 'IGRA_H',
        'area': [
            90, -180, -90,
            180,
        ],
        'format': 'csv-lev.zip',
        'variable': 'air_temperature',
        'pressure_level':500,
        'day': [ '10', ],
        'month': '01',
        'year': '2001',
    },
    'download.csv-lev.zip')
if True:
    # Start Download
    r.download(target='download.zip')
    # Check file size
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    
print("Time elapsed: ", time.time()-t0, "s")

/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2020-11-27 12:10:27,680 INFO Welcome to the CDS
2020-11-27 12:10:27,681 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usag

Time elapsed:  3.069749593734741 s


In [11]:
z = zipfile.ZipFile('download.zip')
print("Unzipping retrieved files")
z.extractall(path='./igra_test')
z.close()
os.remove('download.zip')

Unzipping retrieved files


## ~36.000 values

---

In [12]:
t0 = time.time()

c = cdsapi.Client()
r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'source': 'IGRA_H',
        'area': [
            17, 48, 16,
            49,
        ],
        'format': 'csv-lev.zip',
        'variable': 'air_temperature',
        'pressure_level':500,
        'day': [ '1', ],
        'month': '01',
        'year': '2001',
    },
    'download.csv-lev.zip')
if True:
    # Start Download
    r.download(target='download.zip')
    # Check file size
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    
print("Time elapsed: ", time.time()-t0, "s")

/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2020-11-27 12:12:33,138 INFO Welcome to the CDS
2020-11-27 12:12:33,139 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usag

Time elapsed:  1.3788912296295166 s


In [13]:
t0 = time.time()

c = cdsapi.Client()
r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'source': 'IGRA_H',
        'area': [
            17, 48, 16,
            49,
        ],
        'format': 'csv-lev.zip',
        'variable': 'air_temperature',
        'pressure_level':500,
        'day': [ '2', ],
        'month': '01',
        'year': '2001',
    },
    'download.csv-lev.zip')
if True:
    # Start Download
    r.download(target='download.zip')
    # Check file size
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    
print("Time elapsed: ", time.time()-t0, "s")

/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2020-11-27 12:12:43,950 INFO Welcome to the CDS
2020-11-27 12:12:43,951 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usag

Time elapsed:  1.2588286399841309 s


In [14]:
t0 = time.time()

c = cdsapi.Client()
r = c.retrieve(
    'insitu-observations-igra-baseline-network',
    {
        'source': 'IGRA_H',
        'area': [
            17, 48, 16,
            49,
        ],
        'format': 'csv-lev.zip',
        'variable': 'air_temperature',
        'pressure_level':500,
        'day': [ '10', ],
        'month': '01',
        'year': '2001',
    },
    'download.csv-lev.zip')
if True:
    # Start Download
    r.download(target='download.zip')
    # Check file size
    assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    
print("Time elapsed: ", time.time()-t0, "s")

/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
2020-11-27 12:12:52,359 INFO Welcome to the CDS
2020-11-27 12:12:52,360 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
/jetfs/spack/opt/spack/linux-rhel8-skylake_avx512/gcc-8.3.1/miniconda3-4.8.2-3m7b6t2kgedyr3jnd2nasmgiq7wm27iv/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sis-dev.climate.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usag

Time elapsed:  1.2392616271972656 s


In [15]:
z = zipfile.ZipFile('download.zip')
print("Unzipping retrieved files")
z.extractall(path='./igra_test')
z.close()
os.remove('download.zip')

Unzipping retrieved files


## no data available - empty csv

---